<a href="https://colab.research.google.com/github/GenAIUnplugged/tensorflow_series/blob/main/cifar_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
y_train[0]

array([6], dtype=uint8)

In [ ]:
import tensorflow_datasets as tfds
# Load the CIFAR-10 dataset
ds = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)

In [ ]:
ds

([<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  <_PrefetchDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>],
 tfds.core.DatasetInfo(
     name='cifar10',
     full_name='cifar10/3.0.2',
     description="""
     The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
     """,
     homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
     data_dir='/root/tensorflow_datasets/cifar10/3.0.2',
     file_format=tfrecord,
     download_size=162.17 MiB,
     dataset_size=132.40 MiB,
     features=FeaturesDict({
         'id': Text(shape=(), dtype=string),
         'image': Image(shape=(32, 32, 3), dtype=uint8),
         'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
     }),
     supervised_keys=('image

In [ ]:
(ds_train,ds_test),ds_info = ds

In [ ]:
type(ds_train)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
ds_info

tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_dir='/root/tensorflow_datasets/cifar10/3.0.2',
    file_format=tfrecord,
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=string),
        'image': Image(shape=(32, 32, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,
    },
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Kr

In [ ]:
def preprocess(image, label):
  image = tf.cast(image, tf.float32) / 255.0
  label = tf.one_hot(label, depth=10)
  return image, label

In [ ]:
ds_train = ds_train.map(preprocess)
ds_test = ds_test.map(preprocess)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

BATCH_SIZE = 32

history = model.fit(ds_train.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE),
                    epochs=10,
                    validation_data=ds_test.batch(BATCH_SIZE))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax') # Output layer with 10 units for 10 classes
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 32
history = model.fit(ds_train.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE),
                    epochs=10,
                    validation_data=ds_test.batch(BATCH_SIZE))


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 89s 55ms/step - accuracy: 0.3552 - loss: 1.7433 - val_accuracy: 0.5351 - val_loss: 1.3041
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 72s 46ms/step - accuracy: 0.5677 - loss: 1.2184 - val_accuracy: 0.5937 - val_loss: 1.1527
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 79s 44ms/step - accuracy: 0.6297 - loss: 1.0552 - val_accuracy: 0.6333 - val_loss: 1.0566
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 70s 45ms/step - accuracy: 0.6666 - loss: 0.9479 - val_accuracy: 0.6603 - val_loss: 0.9806
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - accuracy: 0.6952 - loss: 0.8684 - val_accuracy: 0.6797 - val_loss: 0.9374
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 69s 44ms/step - accuracy: 0.7173 - loss: 0.8073 - val_accuracy: 0.6871 - val_loss: 0.9129
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 70s 45ms/step - accuracy: 0.7348 - loss: 0.7570 - val_accuracy: 0.6934 - val_loss: 0.9018
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.7497 -